# CUDA Exercise 05
> Parallelized Vector add.

This Jupyter Notebook can also be open by the google colab, so you don't have to buy a PC with a graphic card to play with CUDA. To launch the Google Colab, please click the below Icon.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg#left)](https://colab.research.google.com/github/SuperChange001/CUDA_Learning/blob/main/Solution/Exercise_05.ipynb)

## Initialize the CUDA dev environment

In [1]:
# clone the code repo,
# !pip install git+git://github.com/depctg/nvcc4jupyter.git
# %load_ext nvcc_plugin
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
# Check the environment
!lsb_release -a
!nvcc --version
!nvidia-smi

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp3nkqsiyn".
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Sat Jun  7 03:12:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG

## Vector Add with Single Thread

In [2]:
%%writefile verctor_add_signal_thread.cu

#include <stdio.h>
#include <assert.h>

#define VECTOR_LENGTH 10000
#define MAX_ERR 1e-4

__global__ void vector_add(float *out, float *a, float *b, int n)
{
    for(int i = 0; i < n; i++)
    {
        out[i] = a[i] + b[i];
    }
}

int main()
{
    float *a, *b, *out;
    float *d_a, *d_b, *d_out;

    // Allocate memory on CPU
    a = (float*)malloc(sizeof(float) * VECTOR_LENGTH);
    b = (float*)malloc(sizeof(float) * VECTOR_LENGTH);
    out = (float*)malloc(sizeof(float) * VECTOR_LENGTH);

    // data initializtion
    for(int i = 0; i < VECTOR_LENGTH; i++)
    {
        a[i] = 3.0f;
        b[i] = 0.14f;
    }

    // Allocate memory on GPU
    cudaMalloc((void**)&d_a, sizeof(float) * VECTOR_LENGTH);
    cudaMalloc((void**)&d_b, sizeof(float) * VECTOR_LENGTH);
    cudaMalloc((void**)&d_out, sizeof(float) * VECTOR_LENGTH);

    // copy operator to GPU
    cudaMemcpy(d_a, a, sizeof(float) * VECTOR_LENGTH, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * VECTOR_LENGTH, cudaMemcpyHostToDevice);

    for(int i=0;i<100;i++)
    {
      // GPU do the work, CPU waits
      vector_add<<<1,1>>>(d_out, d_a, d_b, VECTOR_LENGTH);
    }
    // Get results from the GPU
    cudaMemcpy(out, d_out, sizeof(float) * VECTOR_LENGTH,
               cudaMemcpyDeviceToHost);

    // Test the result
    for(int i = 0; i < VECTOR_LENGTH; i++){
        assert(fabs(out[i] - a[i] - b[i]) < MAX_ERR);
    }
    printf("out[0] = %f\n", out[0]);
    printf("PASSED\n");

    // Free the memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a);
    free(b);
    free(out);
}

Writing verctor_add_signal_thread.cu


## Vector Add with Multiple Threads

In [3]:
%%writefile verctor_add_multi_thread.cu

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>

#define VECTOR_LENGTH 10000
#define MAX_ERR 1e-4

__global__ void vector_add(float *out, float *a, float *b, int n)
{
    int index = threadIdx.x;
    int stride = blockDim.x;
    for(int i = index; i < n; i=i+stride)
    {
        out[i] = a[i] + b[i];
    }
}

int main(int argc, char *argv[])
{
    float *a, *b, *out;
    float *d_a, *d_b, *d_out;
    int list_of_test_block_size[]={1,64,128,256,512,1024};
    int block_size = 1;

    if( argc == 2 ) {
      //printf("The argument supplied is %s\n", argv[1]);
      int arg1 = atoi(argv[1]);  //argv[0] is the program name
                                //atoi = ascii to int

      block_size = list_of_test_block_size[arg1];
    }
    else if( argc > 2 ) {
      printf("Too many arguments supplied.\n");
    }
    else {
      printf("One argument expected.\n");

    }

    printf("The Block size is %d.\n", block_size);

    // Allocate memory on CPU
    a = (float*)malloc(sizeof(float) * VECTOR_LENGTH);
    b = (float*)malloc(sizeof(float) * VECTOR_LENGTH);
    out = (float*)malloc(sizeof(float) * VECTOR_LENGTH);

    // data initializtion
    for(int i = 0; i < VECTOR_LENGTH; i++)
    {
        a[i] = 3.0f;
        b[i] = 0.14f;
    }

    // Allocate memory on GPU
    cudaMalloc((void**)&d_a, sizeof(float) * VECTOR_LENGTH);
    cudaMalloc((void**)&d_b, sizeof(float) * VECTOR_LENGTH);
    cudaMalloc((void**)&d_out, sizeof(float) * VECTOR_LENGTH);

    // copy operator to GPU
    cudaMemcpy(d_a, a, sizeof(float) * VECTOR_LENGTH, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeof(float) * VECTOR_LENGTH, cudaMemcpyHostToDevice);

    for(int i=0;i<100;i++)
    {
      // GPU do the work, CPU waits
      vector_add<<<1,block_size>>>(d_out, d_a, d_b, VECTOR_LENGTH);
    }
    // Get results from the GPU
    cudaMemcpy(out, d_out, sizeof(float) * VECTOR_LENGTH,
               cudaMemcpyDeviceToHost);

    // Test the result
    for(int i = 0; i < VECTOR_LENGTH; i++){
        assert(fabs(out[i] - a[i] - b[i]) < MAX_ERR);
    }
    printf("out[0] = %f\n", out[0]);
    printf("PASSED\n");

    // Free the memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_out);
    free(a);
    free(b);
    free(out);
}

Writing verctor_add_multi_thread.cu


## Evaluation

Measuring the time cost of executing the CUDA fucntion with **signle thread**

In [6]:
!nvcc -o verctor_add_signal_thread verctor_add_signal_thread.cu
!nvprof ./verctor_add_signal_thread
# !./verctor_add_signal_thread

==1394== NVPROF is profiling process 1394, command: ./verctor_add_signal_thread
verctor_add_signal_thread: verctor_add_signal_thread.cu:53: int main(): Assertion `fabs(out[i] - a[i] - b[i]) < MAX_ERR' failed.
==1394== Profiling application: ./verctor_add_signal_thread
==1394== Profiling result:
No kernels were profiled.
No API activities were profiled.
==1394== Warning: Some profiling data are not recorded.
======== Error: Application received signal 134


Measuring the time cost of executing the CUDA fucntion with **multi-threads**

In [7]:
!nvcc -o verctor_add_multi_thread verctor_add_multi_thread.cu
!nvprof ./verctor_add_multi_thread 0
!nvprof ./verctor_add_multi_thread 1
!nvprof ./verctor_add_multi_thread 2
!nvprof ./verctor_add_multi_thread 3
!nvprof ./verctor_add_multi_thread 4
!nvprof ./verctor_add_multi_thread 5

The Block size is 1.
==1478== NVPROF is profiling process 1478, command: ./verctor_add_multi_thread 0
verctor_add_multi_thread: verctor_add_multi_thread.cu:75: int main(int, char**): Assertion `fabs(out[i] - a[i] - b[i]) < MAX_ERR' failed.
==1478== Profiling application: ./verctor_add_multi_thread 0
==1478== Profiling result:
No kernels were profiled.
No API activities were profiled.
==1478== Warning: Some profiling data are not recorded.
======== Error: Application received signal 134
The Block size is 64.
==1489== NVPROF is profiling process 1489, command: ./verctor_add_multi_thread 1
verctor_add_multi_thread: verctor_add_multi_thread.cu:75: int main(int, char**): Assertion `fabs(out[i] - a[i] - b[i]) < MAX_ERR' failed.
==1489== Profiling application: ./verctor_add_multi_thread 1
==1489== Profiling result:
No kernels were profiled.
No API activities were profiled.
==1489== Warning: Some profiling data are not recorded.
======== Error: Application received signal 134
The Block size is